# Benchmarks

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import collections
import time
import os

from creme import base
from creme import compat
from creme import compose
from creme import datasets
from creme import feature_extraction
from creme import linear_model
from creme import metrics
from creme import preprocessing
from creme import optim
from creme import stats
from creme import stream
from keras import layers
from keras import models
from keras import optimizers
import pandas as pd
from sklearn import exceptions
from sklearn import linear_model as sk_linear_model
import torch
import tqdm
from vowpalwabbit import pyvw

Using TensorFlow backend.


In [3]:
def format_ns(d):

    units = collections.OrderedDict({'ns': 1})
    units['μs'] = 1000 * units['ns']
    units['ms'] = 1000 * units['μs']
    units['s'] = 1000 * units['ms']
    units['m'] = 60 * units['s']
    units['h'] = 60 * units['m']
    units['d'] = 24 * units['h']

    parts = []

    for unit in reversed(units):
        amount = units[unit]
        quotient, d = divmod(d, amount)
        if quotient > 0:
            parts.append(f'{quotient}{unit}')
        elif d == 0:
            break

    return ', '.join(parts)


def benchmark(get_X_y, n, get_pp, models, get_metric):

    Result = collections.namedtuple('Result', 'lib model score fit_time pred_time')
    results = []

    for lib, name, model in tqdm.tqdm_notebook(models):

        pp = get_pp()
        metric = get_metric()
        fit_time = 0
        pred_time = 0

        # Determine if predict_one or predict_proba_one should be used in case of a classifier
        pred_func = model.predict_one
        if isinstance(model, base.Classifier) and not metric.requires_labels:
            pred_func = model.predict_proba_one

        for x, y in tqdm.tqdm_notebook(get_X_y(), total=n):

            x = pp.fit_one(x, y).transform_one(x)

            # Predict
            tic = time.perf_counter_ns()
            y_pred = pred_func(x)
            pred_time += time.perf_counter_ns() - tic

            # Score
            metric.update(y_true=y, y_pred=y_pred)

            # Fit
            tic = time.perf_counter_ns()
            model.fit_one(x, y)
            fit_time += time.perf_counter_ns() - tic

        results.append(Result(lib, name, metric.get(), fit_time, pred_time))

    results = pd.DataFrame({
        'Library': [r.lib for r in results],
        'Model': [r.model for r in results],
        metric.__class__.__name__: [r.score for r in results],
        'Fit time': [format_ns(r.fit_time) for r in results],
        'Average fit time': [format_ns(round(r.fit_time / n)) for r in results],
        'Predict time': [format_ns(r.pred_time) for r in results],
        'Average predict time': [format_ns(round(r.pred_time / n)) for r in results]
    })

    return results

In [4]:
class ScikitLearnClassifier(base.MultiClassifier):

    def __init__(self, model, classes):
        self.model = model
        self.classes = classes

    def fit_one(self, x, y):
        self.model.partial_fit([list(x.values())], [y], classes=self.classes)
        return self

    def predict_proba_one(self, x):
        try:
            return dict(zip(self.classes, self.model.predict_proba([list(x.values())])[0]))
        except exceptions.NotFittedError:
            return {c: 1 / len(self.classes) for c in self.classes}
        

class ScikitLearnRegressor(base.Regressor):

    def __init__(self, model):
        self.model = model

    def fit_one(self, x, y):
        self.model.partial_fit([list(x.values())], [y])
        return self

    def predict_one(self, x):
        try:
            return self.model.predict([list(x.values())])[0]
        except exceptions.NotFittedError:
            return 0
        
        
class PyTorchModel:
    
    def __init__(self, network, loss_fn, optimizer):
        self.network = network
        self.loss_fn = loss_fn
        self.optimizer = optimizer

    def fit_one(self, x, y):
        x = torch.FloatTensor(list(x.values()))
        y = torch.FloatTensor([y])

        y_pred = self.network(x)
        loss = self.loss_fn(y_pred, y)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
        return self


class PyTorchRegressor(PyTorchModel, base.Regressor):

    def predict_one(self, x):
        x = torch.FloatTensor(list(x.values()))
        return self.network(x).item()
    
    
class PyTorchBinaryClassifier(PyTorchModel, base.BinaryClassifier):

    def predict_proba_one(self, x):
        
        x = torch.FloatTensor(list(x.values()))
        p = self.network(x).item()
        
        return {True: p, False: 1. - p}
    
    
class KerasModel:

    def __init__(self, model):
        self.model = model

    def fit_one(self, x, y):
        x = [[list(x.values())]]
        y = [[y]]
        self.model.train_on_batch(x, y)
        return self
    
    
class KerasRegressor(KerasModel, base.Regressor):

    def predict_one(self, x):
        x = [[list(x.values())]]
        return self.model.predict_on_batch(x)[0][0]
    

class KerasBinaryClassifier(KerasModel, base.BinaryClassifier):
    
    def predict_proba_one(self, x):
        x = [[list(x.values())]]
        p_true = self.model.predict_on_batch(x)[0][0]
        return {True: p_true, False: 1. - p_true}


class VowpalWabbitRegressor(base.Regressor):

    def __init__(self, **kwargs):
        kwargs['passes'] = 1
        self.model = pyvw.vw('--quiet', **kwargs)

    def format_features(self, x):
        return ' '.join((f'{k}:{v}' for k, v in x.items()))

    def fit_one(self, x, y):
        self.model.learn(f'{y} | {self.format_features(x)}')
        return self

    def predict_one(self, x):
        return self.model.predict(self.format_features(x))

## Bikes (regression)

In [5]:
n_features = 6
lr = 0.005

class PyTorchNet(torch.nn.Module):
    
    def __init__(self, n_features):
        super().__init__()
        self.linear = torch.nn.Linear(n_features, 1)
        torch.nn.init.constant_(self.linear.weight, 0)
        torch.nn.init.constant_(self.linear.bias, 0)
        
    def forward(self, x):
        return self.linear(x)
    
torch_model = PyTorchNet(n_features=n_features)

# Keras
inputs = layers.Input(shape=(n_features,))
predictions = layers.Dense(1, kernel_initializer='zeros', bias_initializer='zeros')(inputs)
keras_model = models.Model(inputs=inputs, outputs=predictions)
keras_model.compile(optimizer=optimizers.SGD(lr=lr), loss='mean_squared_error')


def add_hour(x):
    x['hour'] = x['moment'].hour
    return x

results = benchmark(
    get_X_y=datasets.fetch_bikes,
    n=182470,
    get_pp=lambda: (
        compose.Whitelister('clouds', 'humidity', 'pressure', 'temperature', 'wind') +
        (
            add_hour |
            feature_extraction.TargetAgg(by=['station', 'hour'], how=stats.Mean())
        ) |
        preprocessing.StandardScaler()
    ),
    models=[
        ('creme', 'LinearRegression', linear_model.LinearRegression(
            optimizer=optim.SGD(lr),
            l2=0.,
            intercept_lr=lr
        )),
        ('scikit-learn', 'SGD', ScikitLearnRegressor(
            model=sk_linear_model.SGDRegressor(
                learning_rate='constant',
                eta0=lr,
                penalty='none'
            ),
        )),
        ('PyTorch (CPU)', 'Linear', PyTorchRegressor(
            network=torch_model,
            loss_fn=torch.nn.MSELoss(),
            optimizer=torch.optim.SGD(torch_model.parameters(), lr=lr)
        )),
        ('Keras on Tensorflow (CPU)', 'Linear', KerasRegressor(
            model=keras_model
        )),
    ],
    get_metric=metrics.MSE
)

W0807 21:12:35.161110 140381070276416 deprecation_wrapper.py:119] From /home/max/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0807 21:12:35.192164 140381070276416 deprecation_wrapper.py:119] From /home/max/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0807 21:12:35.241637 140381070276416 deprecation_wrapper.py:119] From /home/max/anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



W0807 21:20:06.917258 140381070276416 deprecation_wrapper.py:119] From /home/max/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:2741: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0807 21:20:06.924174 140381070276416 deprecation_wrapper.py:119] From /home/max/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0807 21:20:06.925892 140381070276416 deprecation_wrapper.py:119] From /home/max/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



In [6]:
results

,Library,Model,MSE,Fit time,Average fit time,Predict time,Average predict time
0,creme,LinearRegression,23.035085,"2s, 622ms, 432μs, 563ns","14μs, 372ns","834ms, 707μs, 565ns","4μs, 574ns"
1,scikit-learn,SGD,25.295369,"36s, 555ms, 976μs, 145ns","200μs, 340ns","14s, 833ms, 859μs, 20ns","81μs, 295ns"
2,scikit-learn,SGD,42.724297,"39s, 930ms, 699μs, 462ns","218μs, 834ns","15s, 781ms, 224μs, 91ns","86μs, 487ns"
3,PyTorch (CPU),Linear,23.035086,"1m, 58s, 351ms, 418μs, 7ns","648μs, 608ns","38s, 722ms, 246μs, 818ns","212μs, 212ns"
4,Keras on Tensorflow (CPU),Linear,23.035086,"4m, 5s, 205ms, 35μs, 827ns","1ms, 343μs, 810ns","2m, 27s, 291ms, 3μs, 275ns","807μs, 207ns"


In [24]:
print(results.to_html(
    index=False,
    columns=['Library', 'MSE', 'Fit time', 'Average fit time', 'Predict time', 'Average predict time'],
    border=0
))

<table border="0" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Library</th>
      <th>MSE</th>
      <th>Fit time</th>
      <th>Average fit time</th>
      <th>Predict time</th>
      <th>Average predict time</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>creme</td>
      <td>23.035085</td>
      <td>2s, 567ms, 982μs, 593ns</td>
      <td>14μs, 73ns</td>
      <td>819ms, 118μs, 167ns</td>
      <td>4μs, 489ns</td>
    </tr>
    <tr>
      <td>scikit-learn</td>
      <td>25.295369</td>
      <td>38s, 692ms, 38μs, 846ns</td>
      <td>212μs, 46ns</td>
      <td>15s, 722ms, 981μs, 63ns</td>
      <td>86μs, 167ns</td>
    </tr>
    <tr>
      <td>PyTorch (CPU)</td>
      <td>23.035086</td>
      <td>1m, 51s, 48ms, 220μs, 315ns</td>
      <td>608μs, 583ns</td>
      <td>35s, 101ms, 306μs, 783ns</td>
      <td>192μs, 368ns</td>
    </tr>
    <tr>
      <td>Keras on Tensorflow (CPU)</td>
      <td>23.035086</td>
      <td>4m, 27s, 475ms, 777μs, 134ns</td

## Electricity prices (binary classification)

In [40]:
n_features = 8
lr = 0.005

# PyTorch
class PyTorchNet(torch.nn.Module):
    
    def __init__(self, n_features):
        super().__init__()
        self.linear = torch.nn.Linear(n_features, 1)
        self.sigmoid = torch.nn.Sigmoid()
        torch.nn.init.constant_(self.linear.weight, 0)
        torch.nn.init.constant_(self.linear.bias, 0)
        
    def forward(self, x):
        return self.sigmoid(self.linear(x))
    
torch_model = PyTorchNet(n_features=n_features)

# Keras
inputs = layers.Input(shape=(n_features,))
predictions = layers.Dense(1, activation='sigmoid', kernel_initializer='zeros', bias_initializer='zeros')(inputs)
keras_model = models.Model(inputs=inputs, outputs=predictions)
keras_model.compile(optimizer=optimizers.SGD(lr=lr), loss='binary_crossentropy')


def add_hour(x):
    x['hour'] = x['moment'].hour
    return x

results = benchmark(
    get_X_y=datasets.fetch_electricity,
    n=45312,
    get_pp=preprocessing.StandardScaler,
    models=[
        ('creme', 'LogisticRegression', linear_model.LogisticRegression(
            optimizer=optim.SGD(lr),
            l2=0.,
            intercept_lr=lr
        )),

        ('scikit-learn', 'SGD', ScikitLearnClassifier(
            model=sk_linear_model.SGDClassifier(
                loss='log',
                learning_rate='constant',
                eta0=lr,
                penalty='none'
            ),
            classes=[False, True]
        )),
        
        ('PyTorch (CPU)', 'Linear', PyTorchBinaryClassifier(
            network=torch_model,
            loss_fn=torch.nn.BCELoss(),
            optimizer=torch.optim.SGD(torch_model.parameters(), lr=lr)
        )),
        
        ('Keras on Tensorflow (CPU)', 'Linear', KerasBinaryClassifier(
            model=keras_model
        )),
        
    ],
    get_metric=metrics.LogLoss
)

In [41]:
results

,Library,Model,LogLoss,Fit time,Average fit time,Predict time,Average predict time
0,creme,LogisticRegression,0.413533,"717ms, 703μs, 959ns","15μs, 839ns","243ms, 460μs, 908ns","5μs, 373ns"
1,scikit-learn,SGD,0.413533,"14s, 350ms, 25μs, 126ns","316μs, 694ns","6s, 51ms, 349μs, 899ns","133μs, 549ns"
2,PyTorch (CPU),Linear,0.418901,"32s, 689ms, 993μs, 527ns","721μs, 442ns","11s, 853ms, 58μs, 557ns","261μs, 588ns"
3,Keras on Tensorflow (CPU),Linear,0.418901,"1m, 32s, 461ms, 383μs, 997ns","2ms, 40μs, 550ns","52s, 576ms, 662μs, 120ns","1ms, 160μs, 325ns"


## Covertype (multi-class)

In [30]:
from sklearn import datasets
from creme import multiclass

n_features = 54
classes = [1, 2, 3, 4, 5, 6, 7]
lr = 0.005

results = benchmark(
    get_X_y=lambda: stream.iter_sklearn_dataset(datasets.fetch_covtype()),
    n=581012,
    get_pp=preprocessing.StandardScaler,
    models=[
        ('sklearn', 'LogisticRegression', ScikitLearnClassifier(
            model=sk_linear_model.SGDClassifier(
                loss='log',
                learning_rate='constant',
                eta0=lr,
                penalty='none'
            ),
            classes=classes
        )),
        
        ('creme', 'LogisticRegression', multiclass.OneVsRestClassifier(
            binary_classifier=linear_model.LogisticRegression(
                optimizer=optim.SGD(lr),
                l2=0.,
                intercept_lr=lr
            )
        )),
    ],
    get_metric=metrics.Accuracy
)

results

KeyboardInterrupt: 